# Day 1

In [8]:
from itertools import pairwise, tee
from pathlib import Path

INPUTS = Path("aoc2021_inputs")

# With lazy iterator:
count = 0
with open(INPUTS / "day1.txt", "r") as f:
    for prev_, next_ in pairwise(map(int, f)):
        count += next_ > prev_
assert count == 1292

def triplewise(it):
    a, b, c = tee(it, 3)
    next(b); next(c); next(c)
    yield from zip(a, b, c)
    
def nwise(it, n):
    its = tee(it, n)
    for i, it in enumerate(its):
        for _ in range(i):
            next(it)
    yield from zip(*its)
    
def triplewise_(it):
    yield from nwise(it, 3)

count = 0
with open(INPUTS / "day1.txt", "r") as f:
    sum_prev = float("+inf")
    for v1, v2, v3 in triplewise_(map(int, f)):
        sum_ = v1 + v2 + v3
        count += sum_ > sum_prev
        sum_prev = sum_
assert count == 1262

# Day 2

In [2]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()

h, v = 0, 0
for line in moves:
    move, size = line.split()

    if move == "forward":
        h += int(size)
    elif move == "up":
        v -= int(size)
    else:
        v += int(size)
        
print(h, v, h * v)
assert h * v == 1727835

1905 907 1727835


In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()
    
horiz, depth, aim = 0, 0, 0
for line in moves:
    move, size = line.split()
    size = int(size)
    
    if move == "forward":
        horiz += size
        depth += size * aim
    elif move == "down":
        aim += size
    else:
        aim -= size
        
print(horiz, depth, aim, horiz * depth)

1905 810499 907 1544000595


# Day 3

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day3.txt")

counter = Counter()
with open(INPUT_PATH, "r") as f:
    for line in f:
        counter += Counter(enumerate(line.strip()))

gamma, epsilon = 0, 0
total_bits = len(counter) // 2
for n in range(total_bits):
    if counter[(n, "1")] > counter[(n, "0")]:
        gamma |= 1 << (total_bits - n - 1)
    else:
        epsilon |= 1 << (total_bits - n - 1)
print(gamma * epsilon)

# --- part 2 ---

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "1"] += 0.5  # Force ending in "1" to win in case of draw.
    (prefix, count), = c.most_common(1)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
oxygen = int(under_consideration[0], 2)
print(oxygen)

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "0"] -= 0.5  # Force ending in "0" to lose in case of draw
    _, (prefix, count) = c.most_common(2)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
co2 = int(under_consideration[0], 2)
print(co2)

print(oxygen * co2)

749376
3871
613
2372923
